In [2]:
"""import self-written functions"""
import utility_functions
import BPE_function

In [3]:
"""read training and test data"""
with open (r"corpora/Shakespeare_clean_train.txt", 'r') as f:
  text_train = f.read()
with open (r"corpora/Shakespeare_clean_test.txt", 'r') as f:
  text_test = f.read()

In [4]:
"""train a dictionary of byte pairs and their respective counts on training and test data"""
#extract words from the text
dict_train = get_words(text_train)
dict_test = get_words(text_test)

#builds the byte pair vocabulary and the sorted token frequency
vocab_train, sorted_token_freq_train, dict_matrix_train = bpe(dict_train)

NameError: name 'get_words' is not defined

In [ ]:
"""test and print performance of training and test set"""
train_accuracy = performance(dict_train, vocab_train, 500)
test_accuracy = performance(dict_test, vocab_train, 500)

print("train accuracy: ", train_accuracy)
print("test accuracy: ", test_accuracy)

In [ ]:
"""compare our BPE split to split of another system (in this case gpt 3.5)"""
with open (file_path, 'r') as f:
  text = f.read()
model_name="gpt-3.5-turbo"
tokens_train = compare_to_gpt_encoding(text_train, vocab_train, model_name)
tokens_test = compare_to_gpt_encoding(text_test, vocab_train, model_name)

In [ ]:
"""run BPE split on different (non-shakespearean) test set and evaluate performance"""
with open ('corpora/friends.txt', 'r') as f:
  friends = f.read()

friends_dict = get_words(friends)
friends_accuracy = performance(friends_dict, vocab_train, 500)
print("Accuracy on Friends: ", friends_accuracy)

#also compare to gpt encoding of BPE split
tokens_friends = compare_to_gpt_encoding(friends, vocab_train, model_name)